# Neurocombat TOP and StrokeMRI harmonized datasets

Note this must be run in the `neurocombaty` environment

## import libraries

In [1]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from neuroCombat import neuroCombat

## Let's see how we would apply this to our data
We will have to flip it on it's side to make it work

In [2]:
filepath = '../open_work/internal_results/cleaned_pvc2s/' 
filename_mri = os.path.join(filepath,'StrokeMRI_pvc2c.csv') 
filename_top = os.path.join(filepath,'TOP_pvc2c.csv') 

In [3]:
our_mri_data = pd.read_csv(filename_mri)
our_top_data = pd.read_csv(filename_top)

In [4]:
print("Our StrokeMRI data is", len(our_mri_data), "patients")
print("Our TOP data is", len(our_top_data), "patients")

Our StrokeMRI data is 514 patients
Our TOP data is 527 patients


In [5]:
our_top_data.head(3)

,Unnamed: 0,participant_id,age,sex,gm_vol,wm_vol,csf_vol,gm_ivc_ratio,gmwm_ivc_ratio,wmh_vol,...,deepwm_b_cov,aca_b_cov,mca_b_cov,pca_b_cov,totalgm_b_cov,deepwm_b,aca_b,mca_b,pca_b,totalgm_b
0,0,sub-0001_1_ses-1_run-1,43.49,M,0.71736,0.52803,0.31812,0.45881,0.45881,1.743,...,8.7620,2.2748,2.3205,2.6858,2.4316,22.1022,80.4744,68.3224,52.4614,60.6981
1,1,sub-0002_1_ses-1_run-1,38.30,F,0.72383,0.62394,0.25673,0.45112,0.45112,1.629,...,9.0749,1.7564,2.3989,1.5982,1.9738,23.5401,87.3972,78.0359,63.9932,71.6047
2,2,sub-0019_1_ses-1_run-1,32.30,M,0.71224,0.53295,0.33594,0.45046,0.45046,0.621,...,8.8791,1.9850,1.8702,2.1648,2.1723,27.5573,94.0855,86.3816,62.6012,74.0588


In [6]:
our_mri_data.head(3)

,Unnamed: 0,participant_id,age,sex,gm_vol,wm_vol,csf_vol,gm_ivc_ratio,gmwm_ivc_ratio,wmh_vol,...,deepwm_b_cov,aca_b_cov,mca_b_cov,pca_b_cov,totalgm_b_cov,deepwm_b,aca_b,mca_b,pca_b,totalgm_b
0,1,sub-59082_1_ses-1_run-1,43.172603,F,0.64754,0.49441,0.31320,0.44500,0.44500,1.249,...,8.0434,1.7273,1.9776,1.7141,1.9140,26.8262,104.3155,90.5047,73.8758,84.4527
1,2,sub-59083_1_ses-1_run-1,66.367123,F,0.60517,0.48594,0.42304,0.39968,0.39968,14.597,...,3.7791,1.9669,1.7154,2.6394,1.8815,24.9597,75.7051,65.3032,65.6771,67.1158
2,4,sub-59085_1_ses-1_run-1,55.838356,F,0.61724,0.53779,0.33692,0.41371,0.41371,6.341,...,5.1248,1.8146,1.9845,2.0827,1.9501,27.7117,81.0966,72.9980,60.8317,64.3183


In [7]:
top_selection = our_top_data.drop(['Unnamed: 0'],axis=1)
mri_selection = our_mri_data.drop(['Unnamed: 0'],axis=1)

In [8]:
#mri_selection.isna().sum()
#top_selection.isna().sum()

In [9]:
mri_selection = mri_selection.set_index('participant_id')
top_selection = top_selection.set_index('participant_id')

In [10]:
top_selection = top_selection.T
mri_selection = mri_selection.T
#mri_selection

In [11]:
both_together = pd.concat([ mri_selection, top_selection], axis=1, join="inner")
both_together

participant_id,sub-59082_1_ses-1_run-1,sub-59083_1_ses-1_run-1,sub-59085_1_ses-1_run-1,sub-59086_1_ses-1_run-1,sub-59087_1_ses-1_run-1,sub-59088_2_ses-2_run-1,sub-59089_1_ses-1_run-1,sub-59089_2_ses-2_run-1,sub-59090_1_ses-1_run-1,sub-59090_2_ses-2_run-1,...,sub-1157_1_ses-1_run-1,sub-1158_1_ses-1_run-1,sub-1159_1_ses-1_run-1,sub-1161_1_ses-1_run-1,sub-1162_1_ses-1_run-1,sub-1163_1_ses-1_run-1,sub-1165_1_ses-1_run-1,sub-1167_1_ses-1_run-1,sub-1168_1_ses-1_run-1,sub-1169_1_ses-1_run-1
age,43.1726,66.3671,55.8384,48.2384,58.6164,46.8164,58.4192,60.9096,72.1014,74.611,...,46.07,46.8,36.76,36.61,40.84,19.06,33.86,33.92,45.31,34.05
sex,F,F,F,F,F,M,F,F,F,F,...,M,M,F,M,M,F,F,M,M,F
gm_vol,0.64754,0.60517,0.61724,0.63402,0.57348,0.64916,0.59539,0.58169,0.54402,0.52956,...,0.64809,0.66598,0.7438,0.77303,0.76223,0.78201,0.71537,0.73473,0.71094,0.67941
wm_vol,0.49441,0.48594,0.53779,0.48542,0.45642,0.51359,0.42689,0.42088,0.43335,0.43469,...,0.53683,0.56342,0.52266,0.57356,0.51381,0.49489,0.48601,0.53549,0.60974,0.50902
csf_vol,0.3132,0.42304,0.33692,0.29969,0.3519,0.35386,0.30742,0.30997,0.42711,0.44153,...,0.35088,0.36952,0.30226,0.24601,0.3145,0.24608,0.27123,0.35985,0.3748,0.30031
gm_ivc_ratio,0.445,0.39968,0.41371,0.44677,0.41502,0.42803,0.44776,0.44318,0.38735,0.3767,...,0.42199,0.41652,0.47414,0.48539,0.47923,0.51347,0.48578,0.45074,0.41931,0.45637
gmwm_ivc_ratio,0.445,0.39968,0.41371,0.44677,0.41502,0.42803,0.44776,0.44318,0.38735,0.3767,...,0.42199,0.41652,0.47414,0.48539,0.47923,0.51347,0.48578,0.45074,0.41931,0.45637
wmh_vol,1.249,14.597,6.341,3.07,1.761,1.559,10.007,15.737,3.91,6.586,...,4.733,1.693,1.144,1.255,3.361,1.431,1.425,3.935,4.17,2.023
wmh_count,24,25,30,23,23,18,63,65,24,28,...,13,25,18,23,20,18,18,20,26,24
deepwm_b_cov,8.0434,3.7791,5.1248,5.0993,6.6565,6.7026,2.5252,1.9384,4.108,3.7965,...,6.2043,6.9749,7.6305,9.8144,7.1005,8.2103,6.6858,4.5665,9.5267,7.0039


In [12]:
# make sure there are no NaNs
both_together.isna().sum().sum()

0

In [13]:
# # save off csv
# both_together.to_csv('both_top_mri_together.csv')

In [14]:
# make and save of csv of features only
features_only = both_together[2:]
features_only.to_csv('features_only_top_mri.csv')

In [15]:
features_only.T.columns

Index(['gm_vol', 'wm_vol', 'csf_vol', 'gm_ivc_ratio', 'gmwm_ivc_ratio',
       'wmh_vol', 'wmh_count', 'deepwm_b_cov', 'aca_b_cov', 'mca_b_cov',
       'pca_b_cov', 'totalgm_b_cov', 'deepwm_b', 'aca_b', 'mca_b', 'pca_b',
       'totalgm_b'],
      dtype='object')

In [16]:
dictionary_features_len = len(features_only.T.columns)
number = 0
made_keys = [] 
made_vals = [] 
for n in features_only.T.columns:
    
    made_keys.append(number)
    made_vals.append(n)
    number +=1

In [17]:
feature_dict = dict(map(lambda i,j : (i,j) , made_keys,made_vals))

In [18]:
ft = features_only.reset_index()
ft = ft.rename(columns={"index": "A"})
ft = ft.drop(['A'], axis=1)
ft = ft.dropna()

In [19]:
bt= both_together.reset_index()
bt = bt.rename(columns={"index": "A"})
bt = bt.drop(['A'], axis=1)
bt = bt.dropna()

In [20]:
bt.head(3)

participant_id,sub-59082_1_ses-1_run-1,sub-59083_1_ses-1_run-1,sub-59085_1_ses-1_run-1,sub-59086_1_ses-1_run-1,sub-59087_1_ses-1_run-1,sub-59088_2_ses-2_run-1,sub-59089_1_ses-1_run-1,sub-59089_2_ses-2_run-1,sub-59090_1_ses-1_run-1,sub-59090_2_ses-2_run-1,...,sub-1157_1_ses-1_run-1,sub-1158_1_ses-1_run-1,sub-1159_1_ses-1_run-1,sub-1161_1_ses-1_run-1,sub-1162_1_ses-1_run-1,sub-1163_1_ses-1_run-1,sub-1165_1_ses-1_run-1,sub-1167_1_ses-1_run-1,sub-1168_1_ses-1_run-1,sub-1169_1_ses-1_run-1
0,43.1726,66.3671,55.8384,48.2384,58.6164,46.8164,58.4192,60.9096,72.1014,74.611,...,46.07,46.8,36.76,36.61,40.84,19.06,33.86,33.92,45.31,34.05
1,F,F,F,F,F,M,F,F,F,F,...,M,M,F,M,M,F,F,M,M,F
2,0.64754,0.60517,0.61724,0.63402,0.57348,0.64916,0.59539,0.58169,0.54402,0.52956,...,0.64809,0.66598,0.7438,0.77303,0.76223,0.78201,0.71537,0.73473,0.71094,0.67941


In [21]:
ft.to_csv('ft_top_mri.csv')

In [22]:
data = np.genfromtxt('ft_top_mri.csv', delimiter=",", skip_header=1)
data

array([[ 0.     ,  0.64754,  0.60517, ...,  0.73473,  0.71094,  0.67941],
       [ 1.     ,  0.49441,  0.48594, ...,  0.53549,  0.60974,  0.50902],
       [ 2.     ,  0.3132 ,  0.42304, ...,  0.35985,  0.3748 ,  0.30031],
       ...,
       [14.     , 90.5047 , 65.3032 , ..., 63.441  , 56.621  , 70.7366 ],
       [15.     , 73.8758 , 65.6771 , ..., 43.3879 , 43.5347 , 46.0536 ],
       [16.     , 84.4527 , 67.1158 , ..., 54.4204 , 51.4417 , 57.9078 ]])

In [23]:
data = data[:, 1:]
#data

In [24]:
# Specifying the batch (scanner variable) as well as a sex covariate to preserve:
first_columns_as_one = [1] * len(mri_selection.columns)
last_columns_as_two = [2] * len(top_selection.columns)
covars = {'batch':first_columns_as_one + last_columns_as_two,
          'sex':both_together.loc['sex',:].values.tolist(),
           'age':both_together.loc['age',:].values.tolist(),} 
covars = pd.DataFrame(covars) 

In [25]:
#covars

In [26]:
covars.shape

(1041, 3)

In [27]:
data.shape

(17, 1041)

In [28]:
# specifify sex as categorical
categorical_cols = ['sex']
# To specify the name of the variable that encodes for the scanner/batch covariate:
batch_col_mine = 'batch'

#Harmonization step:
data_combat = neuroCombat(dat=data,
    covars=covars,
    batch_col=batch_col_mine,
    categorical_cols=categorical_cols)["data"]

[neuroCombat] Creating design matrix
[neuroCombat] Standardizing data across features
[neuroCombat] Fitting L/S model and finding priors
[neuroCombat] Finding parametric adjustments
[neuroCombat] Final adjustment of data


In [29]:
data_combat

array([[ 0.67656987,  0.63600678,  0.64756204, ...,  0.70508168,
         0.68016392,  0.64992675],
       [ 0.50215268,  0.49385397,  0.54465544, ...,  0.52712735,
         0.60290608,  0.50145349],
       [ 0.27484388,  0.3688815 ,  0.29515135, ...,  0.40919701,
         0.42784341,  0.3520809 ],
       ...,
       [99.71138   , 67.85841266, 77.58411205, ..., 61.02720864,
        55.1984502 , 67.97296639],
       [79.90179739, 69.95893115, 64.08273564, ..., 42.46457031,
        42.59253977, 45.65258053],
       [93.4082537 , 71.72525787, 68.22646738, ..., 52.30809302,
        49.75004734, 56.0866537 ]])

In [30]:
data_combat.shape

(17, 1041)

In [31]:
neurocombat = pd.DataFrame(data_combat)
neurocombat

,0,1,2,3,4,5,6,7,8,9,...,1031,1032,1033,1034,1035,1036,1037,1038,1039,1040
0,0.676570,0.636007,0.647562,0.663626,0.605668,0.680628,0.626644,0.613528,0.577465,0.563621,...,0.614335,0.633073,0.717369,0.745197,0.733885,0.757390,0.687591,0.705082,0.680164,0.649927
1,0.502153,0.493854,0.544655,0.493344,0.464931,0.522262,0.435998,0.430110,0.442327,0.443640,...,0.528495,0.555632,0.515374,0.565981,0.505001,0.487033,0.477970,0.527127,0.602906,0.501453
2,0.274844,0.368881,0.295151,0.263278,0.307976,0.319630,0.269895,0.272079,0.372366,0.384711,...,0.398009,0.421258,0.354513,0.267210,0.352634,0.284442,0.315811,0.409197,0.427843,0.352081
3,0.464791,0.423664,0.436396,0.466397,0.437585,0.447869,0.467296,0.463140,0.412475,0.402810,...,0.397182,0.391029,0.453789,0.468496,0.461567,0.498028,0.466881,0.429521,0.394168,0.433800
4,0.464791,0.423664,0.436396,0.466397,0.437585,0.447869,0.467296,0.463140,0.412475,0.402810,...,0.397182,0.391029,0.453789,0.468496,0.461567,0.498028,0.466881,0.429521,0.394168,0.433800
5,-0.741153,8.895530,2.935053,0.573531,-0.371511,0.555642,5.581747,9.718561,1.179975,3.111932,...,9.757454,0.026380,6.762887,-1.375663,5.365667,7.681578,7.662372,7.203047,7.955285,9.576576
6,21.019972,21.780841,25.585188,20.259102,20.259102,17.354338,50.693879,52.215618,21.019972,24.063449,...,7.743664,30.197013,20.376256,26.454788,20.841451,20.376256,20.376256,20.841451,32.068126,31.602931
7,8.940170,5.230700,6.401311,6.379128,7.733720,7.780655,4.139946,3.629495,5.516807,5.245837,...,4.642914,5.574062,6.377191,9.005146,5.725829,7.077788,5.235671,2.663893,8.657506,5.620045
8,1.679010,1.881776,1.752890,1.623410,1.925444,1.925000,1.809928,1.768884,1.785217,1.716077,...,2.031684,1.777919,1.736658,2.025160,1.859267,2.091085,1.483916,1.894825,1.813732,2.016132
9,1.972876,1.733382,1.979178,1.931316,1.878704,1.944880,1.821890,1.687255,2.010782,1.861623,...,2.000205,2.033181,1.947493,1.965557,1.894592,2.139222,1.949944,2.021260,1.939711,2.175206


In [32]:
topper = bt.head(2)

In [33]:
topper = topper.rename_axis(None, axis="columns")

In [34]:
topper

,sub-59082_1_ses-1_run-1,sub-59083_1_ses-1_run-1,sub-59085_1_ses-1_run-1,sub-59086_1_ses-1_run-1,sub-59087_1_ses-1_run-1,sub-59088_2_ses-2_run-1,sub-59089_1_ses-1_run-1,sub-59089_2_ses-2_run-1,sub-59090_1_ses-1_run-1,sub-59090_2_ses-2_run-1,...,sub-1157_1_ses-1_run-1,sub-1158_1_ses-1_run-1,sub-1159_1_ses-1_run-1,sub-1161_1_ses-1_run-1,sub-1162_1_ses-1_run-1,sub-1163_1_ses-1_run-1,sub-1165_1_ses-1_run-1,sub-1167_1_ses-1_run-1,sub-1168_1_ses-1_run-1,sub-1169_1_ses-1_run-1
0,43.1726,66.3671,55.8384,48.2384,58.6164,46.8164,58.4192,60.9096,72.1014,74.611,...,46.07,46.8,36.76,36.61,40.84,19.06,33.86,33.92,45.31,34.05
1,F,F,F,F,F,M,F,F,F,F,...,M,M,F,M,M,F,F,M,M,F


In [35]:
topper = topper.reset_index(drop=False)
topper = topper.rename(columns={"index": "char"})

In [36]:
topper['char'][0] = 'age'
topper['char'][1] = 'sex'

C:\Users\makeda\.conda\envs\neurocombaty\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\makeda\.conda\envs\neurocombaty\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [37]:
topper

,char,sub-59082_1_ses-1_run-1,sub-59083_1_ses-1_run-1,sub-59085_1_ses-1_run-1,sub-59086_1_ses-1_run-1,sub-59087_1_ses-1_run-1,sub-59088_2_ses-2_run-1,sub-59089_1_ses-1_run-1,sub-59089_2_ses-2_run-1,sub-59090_1_ses-1_run-1,...,sub-1157_1_ses-1_run-1,sub-1158_1_ses-1_run-1,sub-1159_1_ses-1_run-1,sub-1161_1_ses-1_run-1,sub-1162_1_ses-1_run-1,sub-1163_1_ses-1_run-1,sub-1165_1_ses-1_run-1,sub-1167_1_ses-1_run-1,sub-1168_1_ses-1_run-1,sub-1169_1_ses-1_run-1
0,age,43.1726,66.3671,55.8384,48.2384,58.6164,46.8164,58.4192,60.9096,72.1014,...,46.07,46.8,36.76,36.61,40.84,19.06,33.86,33.92,45.31,34.05
1,sex,F,F,F,F,F,M,F,F,F,...,M,M,F,M,M,F,F,M,M,F


In [38]:
bottom = neurocombat.reset_index(drop=False)
bottom = bottom.rename(columns={"index": "char"})
bottom.columns = topper.columns
#bottom = bottom.set_index('char', inplace=True)
#set_index('Name', inplace=True)
bottom

,char,sub-59082_1_ses-1_run-1,sub-59083_1_ses-1_run-1,sub-59085_1_ses-1_run-1,sub-59086_1_ses-1_run-1,sub-59087_1_ses-1_run-1,sub-59088_2_ses-2_run-1,sub-59089_1_ses-1_run-1,sub-59089_2_ses-2_run-1,sub-59090_1_ses-1_run-1,...,sub-1157_1_ses-1_run-1,sub-1158_1_ses-1_run-1,sub-1159_1_ses-1_run-1,sub-1161_1_ses-1_run-1,sub-1162_1_ses-1_run-1,sub-1163_1_ses-1_run-1,sub-1165_1_ses-1_run-1,sub-1167_1_ses-1_run-1,sub-1168_1_ses-1_run-1,sub-1169_1_ses-1_run-1
0,0,0.676570,0.636007,0.647562,0.663626,0.605668,0.680628,0.626644,0.613528,0.577465,...,0.614335,0.633073,0.717369,0.745197,0.733885,0.757390,0.687591,0.705082,0.680164,0.649927
1,1,0.502153,0.493854,0.544655,0.493344,0.464931,0.522262,0.435998,0.430110,0.442327,...,0.528495,0.555632,0.515374,0.565981,0.505001,0.487033,0.477970,0.527127,0.602906,0.501453
2,2,0.274844,0.368881,0.295151,0.263278,0.307976,0.319630,0.269895,0.272079,0.372366,...,0.398009,0.421258,0.354513,0.267210,0.352634,0.284442,0.315811,0.409197,0.427843,0.352081
3,3,0.464791,0.423664,0.436396,0.466397,0.437585,0.447869,0.467296,0.463140,0.412475,...,0.397182,0.391029,0.453789,0.468496,0.461567,0.498028,0.466881,0.429521,0.394168,0.433800
4,4,0.464791,0.423664,0.436396,0.466397,0.437585,0.447869,0.467296,0.463140,0.412475,...,0.397182,0.391029,0.453789,0.468496,0.461567,0.498028,0.466881,0.429521,0.394168,0.433800
5,5,-0.741153,8.895530,2.935053,0.573531,-0.371511,0.555642,5.581747,9.718561,1.179975,...,9.757454,0.026380,6.762887,-1.375663,5.365667,7.681578,7.662372,7.203047,7.955285,9.576576
6,6,21.019972,21.780841,25.585188,20.259102,20.259102,17.354338,50.693879,52.215618,21.019972,...,7.743664,30.197013,20.376256,26.454788,20.841451,20.376256,20.376256,20.841451,32.068126,31.602931
7,7,8.940170,5.230700,6.401311,6.379128,7.733720,7.780655,4.139946,3.629495,5.516807,...,4.642914,5.574062,6.377191,9.005146,5.725829,7.077788,5.235671,2.663893,8.657506,5.620045
8,8,1.679010,1.881776,1.752890,1.623410,1.925444,1.925000,1.809928,1.768884,1.785217,...,2.031684,1.777919,1.736658,2.025160,1.859267,2.091085,1.483916,1.894825,1.813732,2.016132
9,9,1.972876,1.733382,1.979178,1.931316,1.878704,1.944880,1.821890,1.687255,2.010782,...,2.000205,2.033181,1.947493,1.965557,1.894592,2.139222,1.949944,2.021260,1.939711,2.175206


In [39]:
back_together = pd.concat([topper, bottom])
back_together = back_together.T
back_together

,0,1,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
char,age,sex,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
sub-59082_1_ses-1_run-1,43.1726,F,0.67657,0.502153,0.274844,0.464791,0.464791,-0.741153,21.02,8.94017,1.67901,1.97288,1.67734,1.91063,28.2889,112.408,99.7114,79.9018,93.4083
sub-59083_1_ses-1_run-1,66.3671,F,0.636007,0.493854,0.368881,0.423664,0.423664,8.89553,21.7808,5.2307,1.88178,1.73338,2.45699,1.88187,23.812,78.76,67.8584,69.9589,71.7253
sub-59085_1_ses-1_run-1,55.8384,F,0.647562,0.544655,0.295151,0.436396,0.436396,2.93505,25.5852,6.40131,1.75289,1.97918,1.98792,1.94257,30.4129,85.1008,77.5841,64.0827,68.2265
sub-59086_1_ses-1_run-1,48.2384,F,0.663626,0.493344,0.263278,0.466397,0.466397,0.573531,20.2591,6.37913,1.62341,1.93132,1.94511,1.77825,23.6758,78.719,74.5791,57.2094,65.1921
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sub-1163_1_ses-1_run-1,19.06,F,0.75739,0.487033,0.284442,0.498028,0.498028,7.68158,20.3763,7.07779,2.09108,2.13922,2.49179,2.18759,28.2919,110.343,99.3748,78.8464,89.183
sub-1165_1_ses-1_run-1,33.86,F,0.687591,0.47797,0.315811,0.466881,0.466881,7.66237,20.3763,5.23567,1.48392,1.94994,1.46119,1.83739,25.8406,83.3842,80.7644,63.4185,72.5711
sub-1167_1_ses-1_run-1,33.92,M,0.705082,0.527127,0.409197,0.429521,0.429521,7.20305,20.8415,2.66389,1.89482,2.02126,2.16997,1.94494,22.581,66.1395,61.0272,42.4646,52.3081
sub-1168_1_ses-1_run-1,45.31,M,0.680164,0.602906,0.427843,0.394168,0.394168,7.95528,32.0681,8.65751,1.81373,1.93971,1.835,1.78044,22.2756,60.0468,55.1985,42.5925,49.75


In [40]:


new_header = back_together.iloc[0] #grab the first row for the header
#back_together = back_together[1:] #take the data less the header row
back_together.columns = new_header #set the header row as the df header
back_together = back_together[1:]
back_together


char,age,sex,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
sub-59082_1_ses-1_run-1,43.1726,F,0.67657,0.502153,0.274844,0.464791,0.464791,-0.741153,21.02,8.94017,1.67901,1.97288,1.67734,1.91063,28.2889,112.408,99.7114,79.9018,93.4083
sub-59083_1_ses-1_run-1,66.3671,F,0.636007,0.493854,0.368881,0.423664,0.423664,8.89553,21.7808,5.2307,1.88178,1.73338,2.45699,1.88187,23.812,78.76,67.8584,69.9589,71.7253
sub-59085_1_ses-1_run-1,55.8384,F,0.647562,0.544655,0.295151,0.436396,0.436396,2.93505,25.5852,6.40131,1.75289,1.97918,1.98792,1.94257,30.4129,85.1008,77.5841,64.0827,68.2265
sub-59086_1_ses-1_run-1,48.2384,F,0.663626,0.493344,0.263278,0.466397,0.466397,0.573531,20.2591,6.37913,1.62341,1.93132,1.94511,1.77825,23.6758,78.719,74.5791,57.2094,65.1921
sub-59087_1_ses-1_run-1,58.6164,F,0.605668,0.464931,0.307976,0.437585,0.437585,-0.371511,20.2591,7.73372,1.92544,1.8787,1.91958,1.91063,8.38123,64.1728,53.3251,43.9044,49.2158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sub-1163_1_ses-1_run-1,19.06,F,0.75739,0.487033,0.284442,0.498028,0.498028,7.68158,20.3763,7.07779,2.09108,2.13922,2.49179,2.18759,28.2919,110.343,99.3748,78.8464,89.183
sub-1165_1_ses-1_run-1,33.86,F,0.687591,0.47797,0.315811,0.466881,0.466881,7.66237,20.3763,5.23567,1.48392,1.94994,1.46119,1.83739,25.8406,83.3842,80.7644,63.4185,72.5711
sub-1167_1_ses-1_run-1,33.92,M,0.705082,0.527127,0.409197,0.429521,0.429521,7.20305,20.8415,2.66389,1.89482,2.02126,2.16997,1.94494,22.581,66.1395,61.0272,42.4646,52.3081
sub-1168_1_ses-1_run-1,45.31,M,0.680164,0.602906,0.427843,0.394168,0.394168,7.95528,32.0681,8.65751,1.81373,1.93971,1.835,1.78044,22.2756,60.0468,55.1985,42.5925,49.75


In [41]:
back_together.head(514).tail(10)

char,age,sex,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
sub-59435_2_ses-2_run-1,66.8219,F,0.602356,0.487544,0.338086,0.421041,0.421041,0.593746,16.4548,6.57799,1.74104,1.89578,1.67717,1.78577,35.8365,99.0372,86.5686,66.8433,75.9356
sub-59437_1_ses-1_run-1,65.3918,F,0.61035,0.480039,0.373393,0.41595,0.41595,3.98694,19.4982,5.83762,2.24212,2.01307,1.65762,1.99248,18.4689,82.6481,72.5633,46.9517,61.8702
sub-59437_2_ses-2_run-1,66.2164,F,0.608253,0.485134,0.375345,0.413346,0.413346,3.72704,17.2156,6.0711,2.32564,2.1097,1.76758,2.19494,22.5971,74.2026,64.8315,43.9747,54.1272
sub-59439_1_ses-1_run-1,75.8521,F,0.550237,0.476443,0.319466,0.407774,0.407774,7.10147,15.6939,5.49262,1.94279,1.87816,1.71264,1.71471,12.6493,74.5853,59.4123,51.8688,55.6553
sub-59439_2_ses-2_run-1,76.8301,F,0.520512,0.484164,0.342127,0.386094,0.386094,8.90203,16.4548,4.66701,1.82575,1.91424,1.67565,1.72374,15.4527,80.8409,56.9465,56.442,57.3042
sub-59440_1_ses-1_run-1,73.9288,M,0.607324,0.535029,0.461714,0.38038,0.38038,18.8861,52.3543,3.91417,1.96156,2.01594,1.84916,2.06418,31.4653,88.0484,78.422,57.6894,63.4419
sub-59440_2_ses-2_run-1,74.7699,M,0.609899,0.517833,0.46906,0.383147,0.383147,15.9738,28.0065,3.9808,2.42861,2.58435,2.14365,2.54591,33.3508,88.1053,83.6165,71.1141,66.5703
sub-59441_2_ses-2_run-1,74.5123,M,0.62088,0.485285,0.383952,0.416416,0.416416,3.21966,26.4848,5.28964,1.79332,1.87975,1.6445,1.76871,29.5863,68.478,67.2549,55.6886,57.1555
sub-59442_1_ses-1_run-1,67.526,M,0.671179,0.560238,0.35623,0.42471,0.42471,5.6476,10.5065,6.23895,1.65589,1.98206,2.06605,1.8749,22.1816,75.2771,68.928,60.2441,63.2406
sub-59442_2_ses-2_run-1,68.411,M,0.661414,0.546737,0.385416,0.416607,0.416607,5.88729,15.0717,5.63603,1.68068,2.10272,2.03007,1.92498,28.3371,87.9254,75.5927,62.9977,69.5364


In [42]:
back_together.tail(527).head(10)

char,age,sex,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
sub-0001_1_ses-1_run-1,43.49,M,0.686888,0.519514,0.357149,0.438598,0.438598,0.18643,28.3259,7.73349,2.44687,2.35871,3.02191,2.51152,23.0459,76.8934,65.1991,50.3742,57.6993
sub-0002_1_ses-1_run-1,38.3,F,0.696452,0.61874,0.297726,0.427895,0.427895,8.31538,29.7318,8.12252,1.83092,2.45261,1.60949,1.98865,24.2158,83.6885,74.2114,61.291,67.8493
sub-0019_1_ses-1_run-1,32.3,M,0.681526,0.524535,0.379375,0.429206,0.429206,-3.4051,7.74366,7.87498,2.07619,1.85705,2.34888,2.20773,27.0964,88.9542,80.6335,59.2133,69.1731
sub-0020_1_ses-1_run-1,21.97,F,0.640511,0.425481,0.30148,0.473023,0.473023,5.45687,29.7318,4.65833,1.97073,1.90705,1.66374,1.87102,26.9194,95.1447,85.0643,64.8546,74.2971
sub-0022_1_ses-1_run-1,37.52,F,0.60515,0.460997,0.25371,0.45993,0.45993,9.53176,27.8607,6.16646,1.63715,2.05088,1.86281,1.77753,25.6492,83.1563,73.8028,65.6921,69.3474
sub-0027_1_ses-1_run-1,33.35,F,0.649623,0.486624,0.309313,0.451078,0.451078,7.7456,31.6029,6.28814,1.74484,1.80734,2.16755,1.94565,27.2455,87.638,80.8361,60.3187,71.1417
sub-0029_1_ses-1_run-1,43.35,M,0.695058,0.575126,0.64388,0.362077,0.362077,-0.629827,20.8415,3.2016,1.87487,2.02237,2.3286,1.98747,25.0471,83.4706,76.7298,59.9716,66.387
sub-0030_1_ses-1_run-1,32.65,M,0.723265,0.558327,0.441601,0.418981,0.418981,-0.719456,35.8104,8.7462,2.13925,2.04064,2.75851,2.36835,24.5433,87.6761,83.7122,62.008,70.0764
sub-0031_1_ses-1_run-1,24.18,M,0.738044,0.610081,0.36018,0.42843,0.42843,0.269657,24.5837,7.9986,2.73811,2.47658,2.06714,2.43385,22.6939,70.8173,63.637,43.2437,55.0165
sub-0032_1_ses-1_run-1,19.32,M,0.810911,0.594639,0.230029,0.491026,0.491026,5.76579,18.9703,8.70318,2.00073,2.14537,1.46426,1.91131,22.6833,87.2783,74.4424,55.73,67.1499


In [43]:
neuro_harm_top =back_together.tail(527)
neuro_harm_mri =back_together.head(514)

In [44]:
neuro_harm_top = neuro_harm_top.rename(feature_dict, axis='columns')
neuro_harm_mri = neuro_harm_mri.rename(feature_dict, axis='columns')

In [46]:
neuro_harm_mri.to_csv('neuro_harm_mri.csv')
neuro_harm_top.to_csv('neuro_harm_top.csv')